In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
asrt_list AS (
    SELECT DISTINCT 
        a.store_id, 
        CASE
            WHEN c.business_unit = '1500' THEN 'Supermarket'
            WHEN c.business_unit = '2000' AND c.concept IN ('WiN','Rural') THEN c.concept
            ELSE 'Urban'
        END business_model,
        a.product_id,
        b.manufacturer_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.product_id = b.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
    WHERE c.business_unit IN ('1500', '2000', '6000')
        AND c.store_status_desc = "Đang hoạt động"
        AND UPPER(c.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(c.store_name) NOT LIKE 'OUTLET%' 
        AND UPPER(c.store_name) NOT LIKE '%HUB%' 
        AND UPPER(c.store_name) NOT LIKE '%ONLINE%'
        AND UPPER(c.store_name) NOT LIKE '%KHO %'
        AND UPPER(c.store_name) NOT LIKE '%B2B%'
        AND a.store_id NOT LIKE 'GH%'
),
d_region_dc AS (
    SELECT 
        dc_id,
        region
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_region_dc
),
dc_sku AS (
    SELECT DISTINCT
        c.dc_id,
        a.product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_vendor_sku a
        INNER JOIN d_region_dc c ON a.region = c.region
),
bu_city_mnf_bz_model_asm_dc_re_model AS (
    SELECT DISTINCT 
        a.business_unit,
        a.city,
        a.manufacturer_id,
        IF(c.sub_manufacturer_id IS NOT NULL, c.sub_manufacturer_id, b.sub_manufacturer_id) sub_manufacturer_id,
        IF(c.sub_manufacturer_id IS NOT NULL, c.dc_id, a.dc_id) dc_id,
        IF(c.sub_manufacturer_id IS NOT NULL, c.replenishment_mode, a.replenishment_mode) replenishment_mode,
        IF(c.sub_manufacturer_id IS NOT NULL, c.xd_dc_id, a.xd_dc_id) xd_dc_id
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_bu_city_mnf_dc_re_mode a 
    LEFT JOIN (
        SELECT DISTINCT manufacturer_id, sub_manufacturer_id
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product
    ) b
        USING(manufacturer_id)
    LEFT JOIN {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_bu_city_mnf_submnf_dc_re_mode c -- Table to be created.
        ON a.business_unit = c.business_unit 
            AND a.city = c.city 
            AND a.manufacturer_id = c.manufacturer_id 
            AND b.sub_manufacturer_id = c.sub_manufacturer_id
),
dc_sku_active AS (
    SELECT DISTINCT 
        a.store_id AS dc_id,
        a.product_id
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
    WHERE b.business_unit = '1100'
),
main AS (
    SELECT DISTINCT
        a1.dc_id,
        a1.xd_dc_id,
        b.business_unit,
        a.store_id,
        b.store_name,
        b.is_win_plus,
        a.business_model,
        b.region_domain_vn AS mien,
        b.city,
        c.mch2_id,
        a.product_id,
        c.product_name,
        a.manufacturer_id,
        c.manufacturer_name,
        d.core_range AS assortment_type,
        a1.replenishment_mode,
        IF(a3.dc_id IS NULL, 'INACTIVE', 'ACTIVE') dc_sku_status
    FROM asrt_list a 
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product c ON a.product_id = c.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment d ON
            a.store_id = d.store_id AND
            a.product_id = d.product_id
        LEFT JOIN bu_city_mnf_bz_model_asm_dc_re_model a1 ON    
            b.business_unit = a1.business_unit AND
            b.city = a1.city AND
            c.manufacturer_id = a1.manufacturer_id AND
            c.sub_manufacturer_id = a1.sub_manufacturer_id
        LEFT JOIN dc_sku_active a3 ON a1.dc_id = a3.dc_id AND a.product_id = a3.product_id
        LEFT JOIN dc_sku a4 ON a1.dc_id = a4.dc_id AND a.product_id = a4.product_id
    WHERE a1.dc_id IS NOT NULL AND a4.product_id IS NOT NULL
),
win_plus AS (
    SELECT
        IF(a.dc_id = '1312' AND a.is_win_plus = 'yes', '1340', a.dc_id) AS dc_id,
        a.* EXCEPT(dc_id)
    FROM main a
),
buildstock_golive AS ( 
    SELECT
        IF(b.dc_bs_id IS NULL, a.dc_id, b.dc_bs_id) AS dc_id,
        IF(b.dc_bs_id IS NULL, a.replenishment_mode, 'bs') AS replenishment_mode,
        a.* EXCEPT(dc_id, replenishment_mode)
    FROM win_plus a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_buildstock_golive b USING (dc_id, product_id, business_unit)
),
flowthrough_golive AS (
    SELECT
        IF(b.dc_ft_id IS NULL, a.dc_id, b.dc_ft_id) dc_id,
        IF(b.dc_ft_id IS NULL, a.replenishment_mode, 'FT') replenishment_mode,
        a.* EXCEPT(dc_id, replenishment_mode)
    FROM buildstock_golive a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive b USING (dc_id, product_id, business_unit)
)
SELECT 
    a.dc_id,
    b.store_name dc_name,
    a.* EXCEPT(dc_id)
FROM flowthrough_golive a
LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.dc_id = b.store_id;
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_his(
 calday DATE,
 dc_id STRING, 
 xd_dc_id STRING,
 business_unit STRING,
 store_id STRING, 
 business_model STRING,
 mch2_id STRING, 
 product_id STRING, 
 assortment_type STRING, 
 manufacturer_id STRING,
 replenishment_mode STRING,
 dc_sku_status STRING
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
) 
""")

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_his 
    WHERE CALDAY = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc_his
SELECT
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS calday,
    dc_id,
    xd_dc_id,
    business_unit,
    store_id,
    business_model,
    mch2_id,
    product_id,
    assortment_type,
    manufacturer_id,
    replenishment_mode,
    dc_sku_status
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_sku_by_dc;
""")

In [0]:
dbutils.notebook.exit("success")